In [0]:
dbutils.fs.mounts()

In [0]:
directory_path = "/mnt/opas-prod/opas-source/delete_retriggers"

files = dbutils.fs.ls(directory_path)
for file in files:
    print(file.name)

In [0]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("Read CSVs").getOrCreate()

# Define the folder path
folder_path = "/mnt/opas-prod/opas-source/delete_retriggers"


# Define a UDF to transform the values
def transform_value(value):
    if value.startswith('n'):
        return value[1:]  # Remove the leading 'n'
    return value.replace(':', '_')  # Replace ':' with '_'

# Register the UDF
transform_udf = F.udf(transform_value)


deleted_ids_ds = spark.read.csv(folder_path, header=False, inferSchema=True).distinct()

deleted_ids_ds.display()

In [0]:
deleted_ids_ds = deleted_ids_ds.withColumn("updated_orbis_ids", transform_udf(F.col("_c0"))).select("updated_orbis_ids").distinct()

deleted_ids_ds.display()

In [0]:
deleted_ids_ds.count() ##20372562. -> 12713684

In [0]:

deleted_ids_ds.write.mode("overwrite").saveAsTable("delete_retriggers.ids_from_csv")


In [0]:
delete_retriggers_layer_ids = spark.table("delete_retriggers.layer_ids")

not_yet_deleted_ids = deleted_ids_ds.join(
    delete_retriggers_layer_ids,
    delete_retriggers_layer_ids.Product_orbis_id == deleted_ids_ds.updated_orbis_ids,
    "inner"
)

#left anti :  12712265
#left : 12713684
not_yet_deleted_ids.count()

In [0]:
not_yet_deleted_ids.select("updated_orbis_ids").coalesce(1).write.mode("overwrite").csv("/mnt/opas-prod/opas-source/delete_retriggers/pending_delete_retriggers/")